In [ ]:
# -*- coding: utf-8 -*-

import torch
import pandas as pd
import numpy as np

dtype = torch.float
device = torch.device('cpu')


In [ ]:
rating = np.load('the-movies-dataset/numpy/small_ratings.npy')
users = np.load('the-movies-dataset/numpy/users.npy')[:100]
movies = np.load('the-movies-dataset/numpy/movies.npy')[:100]

users_num, movies_num, k = len(users), len(movies), 5
rating_len = len(rating)

print(rating[:, 2].max())
print(rating[:, 0].max())
print(len(users)-1)
print(rating_len)

In [ ]:
# normalization
rating[:, 2] -= 2.5
rating[:, 2] /= 2.5

# thita
W = torch.randn(users_num, k, device=device, dtype=dtype) / 10
H = torch.randn(movies_num, k, device=device, dtype=dtype) / 10

print(rating[:, 2].max())

# Create Ds

In [ ]:
ds = []
for ui, i, ri in rating:
    for uj, j, rj in rating:
        if ui != uj or i == j:
            continue
        if ri > rj:
            ds.append((int(ui), int(i), int(j)))

ds = list(set(ds))

In [ ]:
ds[np.random.randint(len(ds))]

# Training

In [ ]:
def predict(u, i):
    Wu = W[u].view(1, W[u].size()[0])
    return torch.mv(Wu, H[i])

In [ ]:
def predict_diff(u, i, j):
    return  (predict(u, i) - predict(u, j))[0]

print(predict_diff(0, 0, 1))

In [ ]:
def partial_BPR(x_uij, partial_x):
    exp_x = np.exp(-x_uij)
    return exp_x / (1 + exp_x) * partial_x

In [ ]:
iteration = 100000
lr = 1e-4

def train(W, H, lr=1e-3, rr=0.02):
    for itr in range(iteration):
        u, i, j = ds[np.random.randint(len(ds))]
        x_uij = predict_diff(u, i, j)

        for f in range(k):
            W[u][f] -= lr * (partial_BPR(x_uij, H[i][f] - H[j][f]) + rr * W[u][f])
            H[i][f] -= lr * (partial_BPR(x_uij, W[u][f]) + rr * H[i][f])
            H[j][f] -= lr * (partial_BPR(x_uij, -W[u][f]) + rr * H[f][f])
        
        if itr % 10000 == 0:
            print(W[18])
            print(H[0])
            
    return W, H

W, H = train(W, H, lr)

In [ ]:
print(W[18])
print(H[0])

In [ ]:
print(W[18])
print(H[0])

# Evaluation

In [ ]:
user_id = 18

In [ ]:
Wu = W[user_id].view(1, W[user_id].size()[0])
prediction = list(zip(list(range(movies_num)), torch.mm(Wu, H.t()).tolist()[0]))
prediction.sort(key=lambda x: x[1], reverse=True)

### get user preference

In [ ]:
movie_rates = []
movie_predict_rates = []

for u, i, r in rating:
    if u == user_id:
        movie_rates.append((int(i), r))

### read movies' titles and genres

In [ ]:
import json
movie_data = []
df = pd.read_csv('the-movies-dataset/movies_metadata.csv')

for index, row in df.iloc[:, [3, 8]].iterrows():
    movie_data += [{'title': row['original_title'], 'genres': [x['name'] for x in json.loads(row['genres'].replace('\'', '"'))]}]

### print result

In [ ]:
print('User ', users[user_id])
print('from rating, he/she likes:')
print('%s %25s %43s' % ('movie_id', 'movie_title', 'movie_genres'))
for m, r in movie_rates:
    if r > 0.5:
        mid = movies[m]-1
        print('%8s %25s %43s' % (mid, movie_data[mid]['title'][:24], movie_data[mid]['genres'][:4]))

print('')
print('from rating, he/she might like:')
print('%s %25s %43s' % ('movie_id', 'movie_title', 'movie_genres'))
for m, r in prediction[:5]:
    mid = movies[m]-1
    r = r * 2.5 + 2.5
    print('%8s %25s %43s' % (mid, movie_data[mid]['title'][:24], movie_data[mid]['genres'][:4]))